In [1]:
import numpy as np

I decided to do the matrix implementation because I thought it was different from the implementation in the other notebook.

The following was my initial attempt. I knew that I had to make a link table where the value of each link was divided by the number of outbound links. This would be multiplied by the pagerank values. When I made the array, I put the linking page as the row, and the linked-to page as the column. However, found that this returned the same value for the pagerank, so I knew something was wrong. I tried transposing the link table, and this seemed to provide interesting values.

My intuition about this being the right way was confirmed when I went to check the original [pagerank article](http://ilpubs.stanford.edu:8090/422/1/1999-66.pdf), which also served as my guide for the dampening part.

In [13]:
links = np.array([[0, 0.5, 0.5, 0],
        [0.5, 0, 0, 0.5],
        [0, 1, 0, 0],
        [0.33, 0.33, 0.33, 0]])
ranks = np.array([[0.25],
                  [0.25],
                  [0.25],
                  [0.25]])
np.array([0.3,0.3,0.3])

array([0.3, 0.3, 0.3])

In [14]:
np.matmul(links,ranks)

array([[0.25  ],
       [0.25  ],
       [0.25  ],
       [0.2475]])

In [4]:
links, np.array([0.3,0.3,0.3])

array([[0.  , 0.15, 0.15],
       [0.15, 0.  , 0.15],
       [0.  , 0.3 , 0.  ]])

In [15]:
np.transpose(links)

array([[0.  , 0.5 , 0.  , 0.33],
       [0.5 , 0.  , 1.  , 0.33],
       [0.5 , 0.  , 0.  , 0.33],
       [0.  , 0.5 , 0.  , 0.  ]])

In [18]:
linksT = np.array([[0.  , 0.5 , 0.  , 0.33],
       [0.5 , 0.  , 1.  , 0.33],
       [0.5 , 0.  , 0.  , 0.33],
       [0.  , 0.5 , 0.  , 0.  ]])

In [19]:
np.matmul(linksT,ranks)

array([[0.2075],
       [0.4575],
       [0.2075],
       [0.125 ]])

In [20]:
ranks2 = np.array([[0.2075],
       [0.4575],
       [0.2075],
       [0.125 ]])

In [21]:
np.matmul(linksT,ranks2)

array([[0.27   ],
       [0.3525 ],
       [0.145  ],
       [0.22875]])

In [22]:
links_test = np.array([[0, 0, 1, 0],
        [0.5, 0, 0, 0],
        [0.5, 1, 0, 1],
        [0, 0, 0, 0]])
ranks_test = np.array([[0.25],
                  [0.25],
                  [0.25],
                  [0.25]])

(The full equation with the dampening part)

In [24]:
ranks_test2 = np.matmul(links_test,ranks_test)*0.85 + 0.15

In [25]:
ranks_test2

array([[0.3625 ],
       [0.25625],
       [0.68125],
       [0.15   ]])

In [26]:
class Webpage:
    def __init__(self, name, idx):
        # create a new webpage, with no links
        self.name = name
        self.links = []
        self.idx = idx
        
    def add_link(self, target_page):
        if not target_page in self.links:
            self.links.append(target_page.idx)
            
    def __str__(self):
        return self.name  

In [36]:
class PageRank:
    def __init__(self, pages, damping_factor=0.85, debug=False, supernode=True):
        self.damping_factor = damping_factor
        self.debug=debug
        self.supernode=supernode

        # create a "supernode" that has a link and backlink to every page
        self.pages = pages.copy() # don't update the actual pages
        supernode = Webpage("supernode", len(self.pages))
        
        if self.supernode:
            for page in self.pages:
                page.add_link(supernode)
                supernode.add_link(page)
                
            self.pages.append(supernode)
        
        self.page_ranks = np.ones(len(self.pages)) / len(self.pages)
        
        self.links = np.zeros((len(self.pages), len(self.pages)))
        for v in range(len(self.pages)):
            link_val = 1/len(self.pages[v].links)
            for link in self.pages[v].links:
                self.links[link][v] = link_val
                
    def run_page_rank(self, iterations, step=1):
        for ii in range(iterations):
            if self.debug and ii % step == 0:
                print("\nIteration #" + str(ii))
                self.print_table(show_supernode=True)
            
            self.page_ranks = np.matmul(self.links,self.page_ranks)*self.damping_factor + (1-self.damping_factor)
        
    def print_table(self, show_supernode=False):
        for idx in range(len(self.page_ranks)):
            if str(self.pages[idx]) != "supernode" or show_supernode:
                print(str(self.pages[idx]) + ": " + str(self.page_ranks[idx]))

In [38]:
pages = []
pageA = Webpage("A", 0)
pageB = Webpage("B", 1)
pageC = Webpage("C", 2)
pageD = Webpage("D", 3)

pageA.add_link(pageB)
pageA.add_link(pageC)

pageB.add_link(pageC)

pageC.add_link(pageA)

pageD.add_link(pageC)


pages.append(pageA)
pages.append(pageB)
pages.append(pageC)
pages.append(pageD)

pageRank = PageRank(pages, debug=True, damping_factor=0.85, supernode=False)
pageRank.run_page_rank(10)
pageRank.print_table(show_supernode=True)


Iteration #0
A: 0.25
B: 0.25
C: 0.25
D: 0.25

Iteration #1
A: 0.36250000000000004
B: 0.25625000000000003
C: 0.68125
D: 0.15000000000000002

Iteration #2
A: 0.7290625000000001
B: 0.3040625
C: 0.6493750000000001
D: 0.15000000000000002

Iteration #3
A: 0.7019687500000001
B: 0.45985156250000003
C: 0.8458046875
D: 0.15000000000000002

Iteration #4
A: 0.868933984375
B: 0.4483367187500001
C: 0.9667105468750001
D: 0.15000000000000002

Iteration #5
A: 0.9717039648437501
B: 0.519296943359375
C: 1.027883154296875
D: 0.15000000000000002

Iteration #6
A: 1.0237006811523437
B: 0.5629741850585939
C: 1.1318765869140628
D: 0.15000000000000002

Iteration #7
A: 1.1120950988769533
B: 0.5850727894897461
C: 1.191100846789551
D: 0.15000000000000002

Iteration #8
A: 1.1624357197711181
B: 0.6226404170227051
C: 1.2474522880889891
D: 0.15000000000000002

Iteration #9
A: 1.2103344448756408
B: 0.6440351809027252
C: 1.3007795353720244
D: 0.15000000000000002
A: 1.2556626050662207
B: 0.6643921390721473
C: 1.33932204

In [33]:
pageRank.links

array([[0. , 0. , 1. , 0. ],
       [0.5, 0. , 0. , 0. ],
       [0.5, 1. , 0. , 1. ],
       [0. , 0. , 0. , 0. ]])

It makes the same link table as manually, and returns the same results as in the presentation! It works!